In [ ]:
import glob #filenames and pathnames utility
import os   #operating sytem utility
import warnings

import flowgatenist as flow
#from flowgatenist import gaussian_mixture as nist_gmm
import flowgatenist.batch_process as batch_p

from Bio.Seq import Seq

import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates
#from matplotlib.backends.backend_pdf import PdfPages

import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
#from scipy import special
#from scipy import misc

import cmdstanpy
import gsf_ims_fitness.stan_utility as stan_utility
import pickle

import seaborn as sns
sns.set()

%load_ext autoreload
%autoreload 2

%matplotlib inline

# set global default style:
sns.set_style("white")
sns.set_style("ticks", {'xtick.direction':'in', 'xtick.top':True, 'ytick.direction':'in', 'ytick.right':True, })
#sns.set_style({"axes.labelsize": 20, "xtick.labelsize" : 16, "ytick.labelsize" : 16})

plt.rcParams['axes.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.edgecolor'] = 'k'

Indicate the directory where the notebook is saved:

In [ ]:
notebook_dir = os.getcwd()
notebook_dir

In [ ]:
main_directory = notebook_dir[:notebook_dir.rfind("\\")]
main_directory

In [ ]:
os.chdir(main_directory)
data_directories = glob.glob('*_Cytom*')
data_directories.sort()
data_directories = np.array(data_directories)
data_directories

In [ ]:
#%%time
summaries = []
mean_summaries = []
for direct in data_directories:
    for plate_str in ['plate_1', 'plate_2']:
        os.chdir(main_directory)
        os.chdir(direct)
        try:
            os.chdir(plate_str)
            plate_pickle_file = direct + f'_{plate_str}_summary.frame_pkl'
            
            try:
                plate_layout = pickle.load(open(plate_pickle_file, 'rb'))
                summaries.append(plate_layout)
            except:
                print(f'file not found: {plate_pickle_file}')
        except:
            print(f'plate not found: {direct}, {plate_str}')

In [ ]:
len(summaries)

In [ ]:
full_summary = pd.concat(summaries, ignore_index=True)

In [ ]:
np.unique(full_summary.plasmid)

In [ ]:
clone_des_list = ['A', 'B', 'C', 'D', 'E', 'F']
parent_list = []
variant_list = []
clone_list = []
for x in full_summary['plasmid']:
    clone_str = ''
    for c in clone_des_list:
        if x[-len(c):] == c:
            clone_str = c
            break
    clone_list.append(clone_str)
    
    if len(clone_str) > 0:
        x = x[:-len(clone_str)]
        if x[-1] == '-': x = x[:-1]
    variant_list.append(x)
    
    par_p = x[:x.find('(')] if '(' in x else x
    parent_list.append(par_p)
    
parent_list = [x.rstrip('-') for x in  parent_list]
variant_list = [x.rstrip('-') for x in  variant_list]
full_summary['parent_plasmid'] = parent_list
full_summary['clone'] = clone_list
full_summary['variant'] = variant_list

parent_plasmids = np.unique(full_summary['parent_plasmid'])
parent_plasmids

In [ ]:
np.unique(full_summary.variant)

In [ ]:
date_plate = []
for file in full_summary['coli_file']:
    if file == '':
        date_plate.append('')
    else:
        date = file[:file.find('_')]
        plate = file[file.find('plate'):]
        plate = plate[:plate.find('_')]
        date_plate.append(f'{date}_{plate}')
full_summary['date_plate'] = date_plate
np.unique(date_plate)

In [ ]:
# List of bad replicates (e.g., outliers) to ignore in quantitative analysis
bad_reps_list = [] # [['2021-01-20', 'plate-2', 'pVER-IPTG-002(F161V)B'],]

In [ ]:
bad_rep = []
for ind, row in full_summary.iterrows():
    is_bad = False
    for bc in bad_reps_list:
        if (row.coli_file.startswith(bc[0]))&(bc[1] in row.coli_file)&(row.plasmid==bc[2]):
            is_bad = is_bad | True
    bad_rep.append(is_bad)
full_summary['is_bad_rep'] = bad_rep

In [ ]:
df = full_summary
df = df[~df.is_bad_rep]
np.unique(df.inducerConcentration)

In [ ]:
ligand_list = np.unique(full_summary.inducerId)
ligand_list = ligand_list[ligand_list!='none']
ligand_list

In [ ]:
# Fix non-uniformity in ligand naming:
new_ligand = []
for ligand in full_summary.inducerId:
    if ligand == '4-Et-Guai':
        ligand = '4-ethylguaiacol'
    elif ligand == '4-Eth-Phen':
        ligand = '4-ethylphenol'
    elif (ligand == '4-V-Phen') or (ligand == '4vinylPhenol'):
        ligand = '4-vinylphenol'
    elif (ligand == 'Sorbose'):
        ligand = 'L-sorbose'
    ligand = ligand.replace('Arabitol', 'arabitol')
    ligand = ligand.replace('Ribose', 'ribose')
    ligand = ligand.replace('Eugenol', 'eugenol')
    ligand = ligand.replace('Isoeugenol', 'isoeugenol')
    new_ligand.append(ligand)
full_summary['inducerId'] = new_ligand

In [ ]:
ligand_list = np.unique(full_summary.inducerId)
ligand_list = ligand_list[ligand_list!='none']
ligand_list

In [ ]:
plt.rcParams["figure.figsize"] = [6, 4]
fig, axs = plt.subplots()

axs.hist(np.log10(full_summary.singlet_count), bins=50);
axs.set_yscale('log')

In [ ]:
# Mark any additional outlier points, either manually (by date_plate and well) or by singlet_count cutoff
outlier_points = []

In [ ]:
#full_summary['is_outlier'] = False

count_cutoff = 10**4

outlier_list = []
for ind, row in full_summary.iterrows():
    is_outlier = False #row.is_outlier
    if row.singlet_count<count_cutoff:
        is_outlier = True
    else:
        for point in outlier_points:
            if (row.date_plate==point[0]) and (row.well==point[1]):
                is_outlier = True
    outlier_list.append(is_outlier)
full_summary['is_outlier'] = outlier_list

In [ ]:
df = full_summary
df = df[df.is_outlier]
df

In [ ]:
# Plot cytometry data for every variant to see if there are any remaining bad clones
plt.rcParams["figure.figsize"] = [12, 4]
for var in np.unique(full_summary['variant']):
    fig, axs = plt.subplots(1, 2)
    fig.suptitle(var, size=20)
    fig.suptitle(var, size=20)
    df_0 = full_summary
    df_0 = df_0[df_0.variant==var]
    plate_list = np.unique(df_0.date_plate)
    
    lig_list = np.unique(df_0.inducerId)
    if var == 'pAN-1201':
        lig_list = ['none']
        min_x = 1
    else:
        lig_list = lig_list[lig_list!='none']

        min_x = df_0.inducerConcentration
        min_x = min_x[min_x>0]
        min_x = min(min_x)
    
    for p in plate_list:
        for clone in np.unique(df_0['clone']):
            df_1 = df_0[df_0.date_plate==p]
            df_1 = df_1[df_1['clone']==clone]
            
            for lig, fmt in zip(lig_list, ['-o', '-^', '-v', '->']):
                for ax in axs:
                    df = df_1[(df_1.inducerId==lig)|(df_1.inducerId=='none')]
                    df_non_zero = df_1[(df_1.inducerId==lig)]
                    if len(df_non_zero)>0:
                        label = f'{p}'
                        label = f'{label}.{clone}' if (clone !='') else label
                        label = f'{label}, {lig}'

                        if df.is_bad_rep.iloc[0]:
                            label = f'{label} - bad rep'
                            fillstyle = 'none'
                        else:
                            fillstyle = None
                        x = df.inducerConcentration
                        y = df['mean']
                        ax.plot(x, y, fmt, label=label, fillstyle=fillstyle, ms=12)
                        
                        df = df[df.is_outlier]
                        x = df.inducerConcentration
                        y = df['mean']
                        ax.plot(x, y, fmt[-1], color='k', fillstyle='none', ms=20)
                    
                
    x_label = '[ligand]'
    for ax in axs:
        ax.set_xscale('symlog', linthresh=min_x/4);
        ax.set_xlabel(f'{x_label} (umol/L)', size=14)
    ax.set_yscale('log');
    ax.legend(loc='upper left', bbox_to_anchor= (1.1, 1.1), ncol=1);

In [ ]:
os.chdir(notebook_dir)
file_name = 'Ligafy_cytom_data_all'
with open(f'{file_name}.pkl', 'wb') as file:
    pickle.dump(full_summary, file)
    
full_summary.to_csv(f'{file_name}.csv', index=False)

In [ ]:
len(np.unique(full_summary.variant))

In [ ]:
np.unique(full_summary.variant)

In [ ]:
def init_stan_fit(x_data, y_data, mid_start=None):
    x_max = max(x_data)
    x_min = min(x_data)
    
    if mid_start is None:
        x_non_zero = x[x>0]
        mid_start = np.exp(np.mean(np.log(x_non_zero)))
    
    low = np.mean(y_data[x_data<=x_min])
    high = np.mean(y_data[x_data>=x_max])
    mid = np.random.normal(1, 0.2) * mid_start
    n = np.random.normal(1, 0.2) * 1.1
    sig = np.random.normal(1, 0.2) * 100
    
    return dict(log_low_level=np.log10(low), log_high_level=np.log10(high), 
                log_IC_50=np.log10(mid), sensor_n=n,
                sigma=sig)

In [ ]:
#stan_model_file = "C:\\Users\\djross\\Documents\\Python Scripts\\gsf_ims_fitness\\gsf_ims_fitness\\Stan models\\"
stan_model_file = "Hill equation fit.stan"
# Load Stan model
stan_model = stan_utility.compile_model(stan_model_file)

In [ ]:
print(full_summary['mean'].min(), full_summary['mean'].max())

In [ ]:
df = full_summary
df = df[df.variant=='pAN-1201']

non_fluorescent_mean = df['mean'].mean()
non_fluorescent_mean_err = df['mean'].std()

ok_to_fail_fit = []
non_fluorescent_mean, non_fluorescent_mean_err

In [ ]:
df = full_summary
df = df[df['mean']<non_fluorescent_mean]
np.unique(df.variant)

In [ ]:
df = full_summary
df = df[df.variant!='pAN-1201']
df['mean'].min(), df['mean'].max()

In [ ]:
for var in np.unique(full_summary.variant):
    f = full_summary
    f = f[f['variant']==var]
    f = f[~f.is_bad_rep]
    f = f[~f.is_outlier]
    
    y = f['mean'] - non_fluorescent_mean
    print(var, min(y), max(y))

In [ ]:
gmin = 10
gmax = {}
gmax['pLigify-IemR'] = 30000
gmax['pLigify-LcLacI'] = 150000
gmax['pLigify-SorR'] = 3000
gmax['pLigify-VprR'] = 30000

In [ ]:
df = full_summary
df = df[df.variant!='pAN-1201']
np.unique(df[df['mean']<300].variant)

In [ ]:
ligand_list

In [ ]:
new_fits = True

In [ ]:
# Prior on sensor_n; <gamma> = alpha/beta = 2; std = sqrt(alpha)/beta = 1.2
sensor_n_alpha = 3
sensor_n_beta = 1.5

In [ ]:
sensor_n_alpha/sensor_n_beta, np.sqrt(sensor_n_alpha)/sensor_n_beta

In [ ]:
import logging
cmdstanpy_logger = logging.getLogger("cmdstanpy")
cmdstanpy_logger.disabled = True

In [ ]:
new_fits

In [ ]:
max(full_summary.inducerConcentration)

In [ ]:
%%time
#max_inducer = 10**20

os.chdir(notebook_dir)
pickle_file = 'Ligafy_cytom_Hill_fits_mean.pkl'
model_pickle_file = 'Ligafy_cytom_Hill_fits_stan_model.pkl'
if new_fits:
    cytom_Hill_fits = {}
else:
    pickled_model = pickle.load(open(model_pickle_file, 'rb'))
    cytom_Hill_fits = pickle.load(open(pickle_file, 'rb'))

for i, r in enumerate(np.unique(full_summary.variant)):
#for i, r in enumerate(['pLigafy-D2']):
    stan_iter = 2000
    if r =='pAN-1201': # Edit this line when we measure the zero-fluorescence control
        run_fit = False
    else:
        f = full_summary
        f = f[f['variant']==r]
        f = f[~f.is_bad_rep]
        #f = f[f['inducerConcentration']<max_inducer]
        f = f[~f.is_outlier]
        
        lig_list = np.unique(f.inducerId)
        lig_list = lig_list[lig_list!='none']
        
        for lig in lig_list:
            df = f[(f.inducerId==lig)|(f.inducerId=='none')]
            df_non_zero = f[(f.inducerId==lig)]
            include_fit = (len(df_non_zero)>4)&(~np.all(df['mean'].isnull()))
            if include_fit:
                x = df['inducerConcentration']
                y = df['mean'] - non_fluorescent_mean
                yerr = df['mean_err']
                x = x[~y.isnull()]
                yerr = yerr[~y.isnull()]
                y = y[~y.isnull()]
                
                yerr[x>=500] *= 2

            if (r, lig) in cytom_Hill_fits.keys():
                #print(f"{i},    existing fit: {r}")
                run_fit = False
                stored_fit = cytom_Hill_fits[(r, lig)]
                # Check to see if there is new data
                new_data = False
                len_mismatch = len(x)!=len(stored_fit[0])
                if len_mismatch:
                    new_data = True
                else:
                    x_mismatch = (np.array(x)!=np.array(stored_fit[0])).any()
                    y_mismatch = (np.array(y)!=np.array(stored_fit[1])).any()
                    yerr_mismatch = (np.array(yerr)!=np.array(stored_fit[2])).any()
                    if (x_mismatch or y_mismatch or yerr_mismatch):
                        new_data = True
                if new_data:
                    run_fit = True
                    print(f"         non-matching data, running new fit: {r}")
                '''
                # Or if diagnostics fail
                else:
                    if r not in ok_to_fail_fit:
                        test1 = stan_utility.check_rhat(stored_fit[-1], rhat_threshold=1.1, verbose=False)
                        test2 = stan_utility.check_n_eff(stored_fit[-1], ratio_threshold=0.001, verbose=False)
                        if not (test1 and test2):
                            run_fit = True
                            print(f"         old fit failed diagnostics: rhat test {test1}, n_eff test {test2}, running new fit: {r}")
                            stan_iter = 10000
                '''
                
            else: # if there isn't a fit in the dictionary, run one
                run_fit = True
                
            if run_fit:
                if include_fit:
                    print(f"{i},    running fit for: {r}, {lig}")
                    log_x_min = min(np.log10(x[x>0])) - 0.5
                    log_x_max = np.log10(max(x)) + 1
                    stan_data = dict(x=x, y=y, y_err=yerr, N=len(x),
                                     log_g_min=np.log10(gmin), log_g_max=np.log10(gmax[r]),
                                     log_x_min=log_x_min, log_x_max=log_x_max,
                                     sensor_n_alpha=sensor_n_alpha,
                                     sensor_n_beta=sensor_n_beta)
                    stan_init = init_stan_fit(x, y)
                    stan_fit = stan_model.sample(data=stan_data, 
                                                 iter_warmup=int(stan_iter/2),
                                                 iter_sampling=int(stan_iter/2), 
                                                 inits=stan_init, 
                                                 chains=4,
                                                 adapt_delta=0.99, 
                                                 max_treedepth=15)
                    cytom_Hill_fits[(r, lig)] = (x, y, yerr, stan_fit)
                    
                else:
                    print(f'{i},    no data for {r}')
                
with open(pickle_file, 'wb') as file:
    pickle.dump(cytom_Hill_fits, file)
                
with open(model_pickle_file, 'wb') as file:
    pickle.dump(stan_model, file)

In [ ]:
#Re-run some variants
'''
#for i, r in enumerate(np.unique(full_summary.variant)):
for i, r in enumerate(['pLigafy-D2-6', 'pLigafy-R2']):
    stan_iter = 10000
    run_fit = True
    if r =='pAN-1201': # Edit this line when we measure the zero-fluorescence control
        run_fit = False
    else:
        f = full_summary
        f = f[f['variant']==r]
        f = f[~f.is_bad_rep]
        f = f[f['inducerConcentration']<max_inducer]
        f = f[~f.is_outlier]
        
        if r=='pLigafy-D2-6':
            lig = '1S-TIQ'
        elif r=='pLigafy-R2':
            lig = '1R-TIQ'
            
        df = f[(f.inducerId==lig)|(f.inducerId=='none')]
        include_fit = (len(df)>4)&(~np.all(df['mean'].isnull()))
        if include_fit:
            x = df['inducerConcentration']
            y = df['mean'] - non_fluorescent_mean
            yerr = df['mean_err']
            x = x[~y.isnull()]
            yerr = yerr[~y.isnull()]
            y = y[~y.isnull()]

        if (r, lig) in cytom_Hill_fits.keys():
            #print(f"{i},    existing fit: {r}")
            #run_fit = False
            stored_fit = cytom_Hill_fits[(r, lig)]
            # Check to see if there is new data
            new_data = False
            len_mismatch = len(x)!=len(stored_fit[0])
            if len_mismatch:
                new_data = True
            else:
                x_mismatch = (np.array(x)!=np.array(stored_fit[0])).any()
                y_mismatch = (np.array(y)!=np.array(stored_fit[1])).any()
                yerr_mismatch = (np.array(yerr)!=np.array(stored_fit[2])).any()
                if (x_mismatch or y_mismatch or yerr_mismatch):
                    new_data = True
            if new_data:
                run_fit = True
                print(f"         non-matching data, running new fit: {r}")


        else: # if there isn't a fit in the dictionary, run one
            run_fit = True

        if run_fit:
            if include_fit:
                print(f"{i},    running fit for: {r}, {lig}")
                stan_data = dict(x=x, y=y, y_err=yerr, N=len(x),
                                 log_g_min=np.log10(gmin), log_g_max=np.log10(gmax),
                                 log_x_min=0, log_x_max=4,
                                 sensor_n_alpha=sensor_n_alpha,
                                 sensor_n_beta=sensor_n_beta)
                stan_init = init_stan_fit(x, y)
                stan_fit = stan_model.sample(data=stan_data, 
                                             iter_warmup=int(stan_iter/2),
                                             iter_sampling=int(stan_iter/2), 
                                             inits=stan_init, 
                                             chains=4,
                                             adapt_delta=0.995, 
                                             max_treedepth=15)
                cytom_Hill_fits[(r, lig)] = (x, y, yerr, stan_fit)
                stan_utility.check_all_diagnostics(stan_fit)
                display(stan_fit.summary())

            else:
                print(f'{i},    no data for {r}')
                
with open(pickle_file, 'wb') as file:
    pickle.dump(cytom_Hill_fits, file)
                
with open(model_pickle_file, 'wb') as file:
    pickle.dump(stan_model, file)
''';

In [ ]:
def hill_funct(x, low, high, mid, n):
    return low + (high-low)*( x**n )/( mid**n + x**n )

In [ ]:
hill_header = ['G0', 'Ginf', 'EC50', 'n']

In [ ]:
hill_params = ['log_low_level', 'log_high_level', 'log_IC_50', 'sensor_n', 'log_high_low_ratio']
param_names = ['log_g0', 'log_ginf', 'log_ec50', 'n', 'log_ginf_g0_ratio']
popt_list = []
perr_list = []
var_list = []
lig_list = []
for k in cytom_Hill_fits.keys():
    var = k[0]
    lig = k[1]
    
    var_list.append(var)
    lig_list.append(lig)
    if (var, lig) in cytom_Hill_fits.keys():
        x, y, yerr, stan_fit = cytom_Hill_fits[(var, lig)]
        popt = [ np.mean(stan_fit.stan_variable(param)) for param in hill_params ] #low, high, mid, n
        perr = [ np.std(stan_fit.stan_variable(param)) for param in hill_params ] #low, high, mid, n
    else:
        pass
        #popt = [np.nan]*3
        #perr = [np.nan]*3
    popt_list.append(popt)
    perr_list.append(perr)

variant_table = pd.DataFrame({'variant':var_list, "ligand":lig_list})

for par, x, xerr in zip(param_names, np.array(popt_list).transpose(), np.array(perr_list).transpose()):
    variant_table[par] = x
    variant_table[f'{par}_err'] = xerr

In [ ]:
variant_table

In [ ]:
os.getcwd()

In [ ]:
glob.glob('*.pkl')

In [ ]:
pickle_file = 'Ligafy_cytom_Hill_fit_results_mean.pkl'
with open(pickle_file, 'wb') as file:
    pickle.dump(variant_table, file)

variant_table.to_csv(pickle_file.replace('.pkl', '.csv'), index=False)

In [ ]:
hill_params = ['low_level', 'high_level', 'IC_50', 'sensor_n', 'log_high_low_ratio']
param_names = ['G0', 'Ginf', 'EC50', 'n', 'Ginf_G0_ratio', 'Gmax', 'Gmax_G0_ratio']
popt_list = []
perr_list = []
var_list = []
lig_list = []
for k in cytom_Hill_fits.keys():
    var = k[0]
    lig = k[1]
    
    var_list.append(var)
    lig_list.append(lig)
    if (var, lig) in cytom_Hill_fits.keys():
        x, y, yerr, stan_fit = cytom_Hill_fits[(var, lig)]
        popt = [ np.mean(stan_fit.stan_variable(param)) for param in hill_params[:-1] ] #low, high, mid, n
        perr = [ np.std(stan_fit.stan_variable(param)) for param in hill_params[:-1] ] #low, high, mid, n
        
        param = hill_params[-1]
        popt.append(np.mean(10**stan_fit.stan_variable(param)))
        perr.append(np.std(10**stan_fit.stan_variable(param)))
        
        gmax = stan_fit.stan_variable('max_level')
        popt.append(np.quantile(gmax, 0.05))
        perr.append(np.nan)
        
        g0 = stan_fit.stan_variable('low_level')
        popt.append(np.quantile(gmax/g0, 0.05))
        perr.append(np.nan)
    else:
        pass
        #popt = [np.nan]*3
        #perr = [np.nan]*3
    popt_list.append(popt)
    perr_list.append(perr)

variant_table_2 = pd.DataFrame({'variant':var_list, "ligand":lig_list})

for par, x, xerr in zip(param_names, np.array(popt_list).transpose(), np.array(perr_list).transpose()):
    variant_table_2[par] = x
    variant_table_2[f'{par}_err'] = xerr

In [ ]:
variant_table_2

In [ ]:
save_params = ['G0', 'Ginf', 'EC50', 'Ginf_G0_ratio', 'n']
for p in save_params:
    print(p, max(variant_table_2[f'{p}_err']/variant_table_2[p]))

In [ ]:
pickle_file = 'Ligafy_cytom_Hill_fit_results_mean_non_log.pkl'

variant_table_2.to_csv(pickle_file.replace('.pkl', '.csv'), index=False)

In [ ]:
# Get lower limits on EC50 (for dose-response curves with high EC50):
for k in cytom_Hill_fits.keys():
    var = k[0]
    lig = k[1]
    x, y, yerr, stan_fit = cytom_Hill_fits[(var, lig)]
    ec50 = stan_fit.stan_variable('IC_50')
    if np.mean(ec50) > max(x)/10:
        lim = np.quantile(ec50, 0.05)
        print(f'{var}, {lig}, {lim:.0f}, {max(x)}')

In [ ]:
df = full_summary
x_max = df.inducerConcentration.max()
df = df[df.inducerConcentration>0]
x_min = df.inducerConcentration.min()
x_fit = [0] + list(np.logspace(np.log10(x_min/2), np.log10(x_max*2), 20))

In [ ]:
# Plot data with fits to check fit quality
plt.rcParams["figure.figsize"] = [12, 4]


for var in np.unique(full_summary['variant']):
    df_0 = full_summary
    df_0 = df_0[~df_0.is_bad_rep]
    df_0 = df_0[df_0.variant==var]
        
    lig_list = np.unique(df_0.inducerId)
    lig_list = lig_list[lig_list!='none']
    
    for lig in lig_list:
        df = df_0
        df = df[(df.inducerId==lig)|(df.inducerId=='none')]
        df_non_zer = df[(df.inducerId==lig)]
        
        if ((var, lig) in cytom_Hill_fits):# and (len(df_non_zer)>1):
            fig, axs = plt.subplots(1, 2)
            fig.suptitle(f'{var}, {lig}', size=20)
            
            variant_row = variant_table[(variant_table.variant==var)&(variant_table.ligand==lig)].iloc[0]
            hill_params = [variant_row[p] for p in ['log_g0', 'log_ginf', 'log_ec50', 'n']]
            hill_params = [10**p for p in hill_params[:3]] + hill_params[3:]
            y_fit = hill_funct(x_fit, *hill_params)
            for ax in axs:
                x = df.inducerConcentration
                y = df['mean'] - non_fluorescent_mean
                yerr = df['mean_err']*np.mean(cytom_Hill_fits[(var, lig)][-1].stan_variable('sigma'))
                ax.errorbar(x, y, yerr, fmt='o', ms=10)
                
                ax.plot(x_fit, y_fit)

            for ax in axs:
                ax.set_xscale('symlog', linthresh=min(x[x>0])/4);
                ax.set_xlabel(f'[{lig}] (umol/L)')
            ax.set_yscale('log');
        else:
            #pass
            print(f'No fit for {var}, {lig}')